3.1 Reducción de Dimensionalidad
-----


### Leer Dataset
-----


In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model


Using TensorFlow backend.


In [3]:
from keras.datasets import mnist
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt

In [10]:
from utils.multi_gpu import make_parallel

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#Datos de entrenamiento
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
#Datos de Test
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

#Datos de validacion
x_val = x_train[:5000, :]
x_train = x_train[5000:, :]
y_val = y_train[:5000]
y_train = y_train[5000:]

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
y_val = np_utils.to_categorical(y_val, 10)

### Objetivo
-----


**(a)** Entrene un AE basico (1 capa escondida) para generar una representacion de MNIST en d′ = 2, 8, 32, 64 dimensiones. Justifique la eleccion de la funcion de perdida a utilizar y del criterio de entrenamiento en general. Determine el porcentaje de compresion obtenido y el error de reconstrucci ́on en cada caso. ¿Mejora el resultado si elegimos una funcion de activacion ReLU para el Encoder? ¿Podr ́ıa utilizarse esta activacion en el Decoder?

In [36]:
def simple_AE(x_train_,x_val_,d_, funciones_):
    input_img = Input(shape=(784,))
    encoded = Dense(d_, activation=funciones_[0])(input_img)
    decoded = Dense(784, activation=funciones_[1])(encoded)
    
    autoencoder = Model(input=input_img, output=decoded)
    encoder = Model(input=input_img, output=encoded)
    encoded_input = Input(shape=(32,))
    #decoder_layer = autoencoder.layers[-1]
    #decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

    autoencoder.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
    autoencoder.fit(x_train_, x_train_,epochs=50,batch_size=25,shuffle=True,verbose=0,validation_data=(x_val_, x_val_))
    autoencoder.save('modelos_a/basic_autoenconder_' + str(d_) + funciones_[0] + '_' + funciones_[1] + '.h5')

Dimenciones del input:


In [37]:
dimensiones_input = [2, 8, 32, 64]

Funciones de activación:

In [38]:
funciones_activacion = [('sigmoid', 'sigmoid'), ('relu', 'sigmoid'), ('sigmoid', 'relu')]

In [39]:
for dimension in dimensiones_input:
    for funcion in funciones_activacion:
        simple_AE(x_train,x_val,dimension, funcion)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [40]:
ls modelos_a/

basic_autoenconder_2relu_sigmoid.h5
basic_autoenconder_2sigmoid_relu.h5
basic_autoenconder_2sigmoid_sigmoid.h5
basic_autoenconder_32relu_sigmoid.h5
basic_autoenconder_32sigmoid_relu.h5
basic_autoenconder_32sigmoid_sigmoid.h5
basic_autoenconder_64relu_sigmoid.h5
basic_autoenconder_64sigmoid_relu.h5
basic_autoenconder_64sigmoid_sigmoid.h5
basic_autoenconder_8relu_sigmoid.h5
basic_autoenconder_8sigmoid_relu.h5
basic_autoenconder_8sigmoid_sigmoid.h5


Evaluamos por modelo su performance:

In [62]:
basic_autoenconder_2relu_sigmoid = load_model('modelos_a/basic_autoenconder_2relu_sigmoid.h5')
x_pred = basic_autoenconder_2relu_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  5.60%


In [63]:
basic_autoenconder_2sigmoid_relu = load_model('modelos_a/basic_autoenconder_2sigmoid_relu.h5')
x_pred = basic_autoenconder_2sigmoid_relu.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  56.99%


In [43]:
basic_autoenconder_2sigmoid_sigmoid = load_model('modelos_a/basic_autoenconder_2sigmoid_sigmoid.h5')
x_pred = basic_autoenconder_2sigmoid_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  6.75%


In [61]:
basic_autoenconder_32relu_sigmoid = load_model('modelos_a/basic_autoenconder_32relu_sigmoid.h5')
x_pred = basic_autoenconder_32relu_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  0.97%


In [60]:
basic_autoenconder_32sigmoid_relu = load_model('modelos_a/basic_autoenconder_32sigmoid_relu.h5')
x_pred = basic_autoenconder_32sigmoid_relu.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  33342.51%


In [59]:
basic_autoenconder_32sigmoid_sigmoid = load_model('modelos_a/basic_autoenconder_32sigmoid_sigmoid.h5')
x_pred = basic_autoenconder_32sigmoid_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  3.68%


In [58]:
basic_autoenconder_64relu_sigmoid = load_model('modelos_a/basic_autoenconder_64relu_sigmoid.h5')
x_pred = basic_autoenconder_64relu_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  0.40%


In [57]:
basic_autoenconder_64sigmoid_relu = load_model('modelos_a/basic_autoenconder_64sigmoid_relu.h5')
x_pred = basic_autoenconder_64sigmoid_relu.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  268689.04%


In [56]:
basic_autoenconder_64sigmoid_sigmoid = load_model('modelos_a/basic_autoenconder_64sigmoid_sigmoid.h5')
x_pred = basic_autoenconder_64sigmoid_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  2.01%


In [55]:
basic_autoenconder_8relu_sigmoid = load_model('modelos_a/basic_autoenconder_8relu_sigmoid.h5')
x_pred = basic_autoenconder_8relu_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  3.46%


In [54]:
basic_autoenconder_8sigmoid_relu = load_model('modelos_a/basic_autoenconder_8sigmoid_relu.h5')
x_pred = basic_autoenconder_8sigmoid_relu.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  261333.00%


In [53]:
basic_autoenconder_8sigmoid_sigmoid = load_model('modelos_a/basic_autoenconder_8sigmoid_sigmoid.h5')
x_pred = basic_autoenconder_8sigmoid_sigmoid.predict(x_test)
m,n = x_test.shape
aux = ((x_test - x_pred) ** 2).sum(axis=1) / n
reconstruction_error = aux.sum() / m
print('Error de reconstruccion:  %.2f%%' % (reconstruction_error*100))

Error de reconstruccion:  6.75%


Del los resultados podemos concluir que la incorporación de la no-linealidad con ReLU no tiene resultados satisfactorios, asi tambien existe o se puede evidenciar una correlación entre la dimencionalidad y el error. 